In [1]:
!pip install ~/pfcc-extras-v0.4.0/.
!pip install pfp-api-client matlantis-features

Looking in indexes: https://pypi.matlantis-common.svc:8080/simple
Processing /home/jovyan/pfcc-extras-v0.4.0
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for pfcc-extras, since package 'wheel' is not installed.
  Attempting uninstall: pfcc-extras
    Found existing installation: pfcc-extras 0.4.0
    Uninstalling pfcc-extras-0.4.0:
      Successfully uninstalled pfcc-extras-0.4.0
  Running setup.py install for pfcc-extras ... done
You should consider upgrading via the '/home/jovyan/.py38/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.matlantis-common.svc:8080/simple
You should consider upgrading via the '/home/jovyan/.py38/bin/python -m pip install --upgrade pip' command.


In [3]:
pip install -U pfp-api-client

Looking in indexes: https://pypi.matlantis-common.svc:8080/simple
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pfp-api-client: filename=pfp_api_client-1.8.2-py3-none-any.whl size=31766 sha256=6b820c4fc41fc7bf7ddf0203a9172c906410f63055fb369b91e04b22e80419a4
  Stored in directory: /home/jovyan/.cache/pip/wheels/d1/fc/5d/2faf8ba23f13121426d24b4ac591777411231380dbed0a554f
Successfully built pfp-api-client
  Attempting uninstall: pfp-api-client
    Found existing installation: pfp-api-client 1.7.0
    Uninstalling pfp-api-client-1.7.0:
      Successfully uninstalled pfp-api-client-1.7.0
You should consider upgrading via the '/home/jovyan/.py38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator

estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0_PLUS_D3)
calculator = ASECalculator(estimator)

In [13]:
from ase import Atoms
from ase.calculators.calculator import Calculator
from ase.optimize.optimize import Optimizer
from ase.constraints import ExpCellFilter
from ase.spacegroup.symmetrize import FixSymmetry
from ase.io import write

def optimization(
        atoms_in: Atoms,
        calculator: Calculator,
        optimizer: Optimizer,
        outprefix: str = "out",
        cell: bool = False,
        fix_symmetry: bool = False,
        hydrostatic_strain: bool = False,
        fmax: float = 0.005,
        verbose: bool = True,
    ) -> Atoms:
    ciffile = outprefix + ".cif"
    trajfile = outprefix + ".traj"
    atoms = atoms_in.copy()
    atoms_in.calc = calculator
    atoms.calc = calculator
    log = "-" if verbose else None
    if fix_symmetry:
        atoms.set_constraint([FixSymmetry(atoms)])
    if cell:
        ecf = ExpCellFilter(atoms, hydrostatic_strain=hydrostatic_strain)
        opt = optimizer(ecf, logfile=log, trajectory=trajfile)
    else:
        opt = optimizer(atoms, logfile=log, trajectory=trajfile)
    opt.run(fmax=fmax)

    cell_diff = (atoms.cell.cellpar() / atoms_in.cell.cellpar() - 1.0) * 100
    if verbose:
        print(f"Epot before opt: {atoms_in.get_potential_energy()} eV")
        print(f"Epot after opt: {atoms.get_potential_energy()} eV")
    if cell:
        if verbose:
            print("Optimized Cell         :", atoms.cell.cellpar())
            print("Optimized Cell diff (%):", cell_diff)
    write(ciffile, atoms)
    return atoms

In [9]:
from ase.io import read
from pfcc_extras.visualize.view import view_ngl
atoms = read("closed_273K.cif")
view_ngl(atoms, representations=["ball+stick"])

In [14]:
from ase.optimize import LBFGS
opted = optimization(atoms, calculator, LBFGS, fix_symmetry=True)

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 10:27:48     -468.207305*      10.4486
LBFGS:    1 10:27:48     -477.000831*      11.6361
LBFGS:    2 10:27:48     -486.284625*       5.2070
LBFGS:    3 10:27:48     -488.852720*       2.9724
LBFGS:    4 10:27:49     -490.106841*       1.6957
LBFGS:    5 10:27:49     -491.122271*       0.8905
LBFGS:    6 10:27:49     -491.485035*       0.9111
LBFGS:    7 10:27:49     -491.839683*       0.6501
LBFGS:    8 10:27:49     -492.195347*       0.5394
LBFGS:    9 10:27:49     -492.365654*       0.5197
LBFGS:   10 10:27:49     -492.454582*       0.3995
LBFGS:   11 10:27:49     -492.513648*       0.2554
LBFGS:   12 10:27:49     -492.550630*       0.3179
LBFGS:   13 10:27:49     -492.581620*       0.3388
LBFGS:   14 10:27:49     -492.619281*       0.3164
LBFGS:   15 10:27:50     -492.657996*       0.3251
LBFGS:   16 10:27:50     -492.688405*       0.2455
LBFGS:   17 10:27:50     -492.7076

In [15]:
from ase.io import Trajectory
traj = Trajectory('out.traj')

v = view_ngl(traj, representations=["ball+stick"], replace_structure=True)
display(v)

In [16]:
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
ase2pymat = AseAtomsAdaptor()
def get_symmetrized_structure(ase_structure):
    pymat = ase2pymat.get_structure(ase_structure)
    sg = SpacegroupAnalyzer(pymat)
    return sg.get_symmetrized_structure()
symmetrized = get_symmetrized_structure(opted)
symmetrized.to(filename="opted_closed_sym.cif",symprec=0.01)

/home/jovyan/.py38/lib/python3.8/site-packages/pymatgen/io/ase.py:176: UserWarning: Only FixAtoms is supported by Pymatgen. Other constraints will not be set.
  warnings.warn("Only FixAtoms is supported by Pymatgen. Other constraints will not be set.")
